## ABSTRACT

Contained below is a functional twitter bot that accesses tweets via the Tweepy library and Twitter API. The connection is first intialized using API, Bearer, and Clent keys/IDs. Once verified, two functions are created.

1. Search_Tweets()

Used to scrape all tweets in the explore page that contain one of the keywords specified in the argument list. For this project's purpose those keywords were centered around finding job posts via twitter for jobs in the sports market. This is not a perfect method of finding job postings nor is twitter the ideal platform as many tweets are subjective and opinionated thus resulting in an endless amount of variations of verbiage. In other words, the texts scraped from twitter could use a combination of the words job, hire, and resume for purposes other than actual employment. This function returns three arrays:

    1. tweet_results - Contains information regarding individual tweets that meet the keyword criteria. Some info includes tweet id, tweet date/time, tweet contents and URLS if any. 

    2. twitter_user_results - Contains information regarding the users of each tweet including but not limited to user id/name/username, user location, and user join date.

    3. tweet_mention_results - Contains information regarding users mentioned in tweets in case a recruiter or imortant figure is mentioned. These fields are tweet id, tweet user id, and the username of the mentioned account.

2. User_Timeline()

Defined to pull tweets from all unique users pulled via Search_Tweets(). This function returns an identical array to tweet_results so that the two can be later combined into one table of tweets and tweet info. The one returned field is:

    1. user_tweets - Contains identical info to tweet_results for all tweets by all users gathered in function 1 but only from the past 24 hours. 

Finally a tibble is created containing the value 1 and the string "Twitter". This tibble called Sources is created with future data scraping in mind as job postings can be referenced by a source ID which states where the post was pulled from.

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Once gathered, the data is passed into the SQL database which is created via the Python/MYSQL connection established using pymysql. The execute_query function is the only function defined for this portion to allow each query to be executed and a message to display if succesful.

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

This bot works appropriately however as previously mentioned is not ideal for job searching. Improvements could be expanding the search for greater than 24 hours and to clean out irrelvant or subjective tweets. For an example of a subjective tweet issue read the below scenario:

Scenario 1

If a twitter user did not like an NFL player or coach's performace in a game they may tweet, "Wow what a terrible call by THIS SPECIFIC NFL COACH, they deserve to be out of a Job!! @NFLTEAM let me know when you are hiring for a new head coach because Id like to apply and my resume is astounding!". While that twitter user may not be even remotely qualified nor serious about the tweet, this specific tweet would trigger keywords such as 'Job','hiring','apply', and even 'resume'. This is just one example of how difficult twitter scraping can be when looking for serious objective content.  

In [1]:
## Importing necessary libraries and storing API Keys as a comments for reference


# API KEY: FB7QAHPem56r0Mv83RKuPJR4Y
# API KEY SECRET: r7j3GMhjRIXtXjkNnlypjqQW60jDTkojPBPcieQlahdTHr3Q9h
# BEARER TOKEN: AAAAAAAAAAAAAAAAAAAAAMuRiQEAAAAASfWfjQlkVEO7bLR%2B2LI%2FS8s42Eo%3D4dXlyGNYxVfKk2QD1kW4C3GmDZgSZuuFbF8XYbBaC3Aqi6AZSJ
# Client ID: UkxpeUtFN0pDanNWRS1tSk9ZTXY6MTpjaQ
# Client Secret: wTSnz2QkfIittU70Vw0rhKFm6FTjkTbeMTHzZXUUwHK4rcd7S_

import pandas as pd
import numpy as np
import csv
import os
import datetime, time
import pytz
#!pip install wget
import wget
#!pip install -- tweepy
import tweepy
## !pip install mysql.connector
# import mysql.connector
## !pip install pymysql
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb
import xlrd


In [2]:
## Installation / Access to Twitter Account, Prints Authentification Succesful if connection is valid

consumer_key = 'FB7QAHPem56r0Mv83RKuPJR4Y'
consumer_secret = 'r7j3GMhjRIXtXjkNnlypjqQW60jDTkojPBPcieQlahdTHr3Q9h'
access_token = '1583139235342811138-CNcRKakrtwYsJdRhjNvUqXsrE6mha8'
access_token_secret = '1kpR1zSr9wlE1wk8MexeytDHlTzUgBM31qzNsSmF4mzlX'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit = True)

try:
    api.verify_credentials()
    print("Authentication Succesful")
except:
    print("Error during authentication, check keys and try again!!")

Authentication Succesful


# Once Authenticated Functions are Defined as seen below:

In [3]:
## Function to search for the keywords and retrieve tweet contents and properties as well as user information. Returns two 
## lists one for tweet info and one for user info.

def search_tweets(keywords):
    tweet_results = []
    twitter_user_results = []
    tweet_mention_results = []
    i = 0
    for query in keywords:
        
        for tweet in tweepy.Cursor(api.search_tweets, q = query, count=5,
                              tweet_mode = 'extended').items():
            
            if tweet.full_text.startswith('RT @'):
                pass
                #text = tweet.retweeted_status.full_text
                #tweet_results.append(s + 'RT @' + text)
            else:
                i += 1
                s = (i)

                if(tweet.entities['user_mentions'] == []):
                    if tweet.entities['urls'] == []:
                        tweet_results.append([s, tweet.id, tweet.full_text, tweet.created_at, tweet.place, tweet.retweet_count, tweet.favorite_count, "NO URLS", query])
                
                        twitter_user_results.append([tweet.user.id, tweet.user.screen_name, tweet.user.name, tweet.user.description, 
                                      tweet.user.location, tweet.user.created_at, tweet.user.favourites_count, tweet.user.followers_count, tweet.user.friends_count, 
                                      tweet.user.profile_image_url])

                        tweet_mention_results.append([s, tweet.id,tweet.user.id, 'No Mentions'])
                
                    else:
                        tweet_results.append([s, tweet.id, tweet.full_text, tweet.created_at, tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.entities['urls'][0]['url'], query])
                
                        twitter_user_results.append([tweet.user.id, tweet.user.screen_name, tweet.user.name, tweet.user.description, 
                                      tweet.user.location, tweet.user.created_at, tweet.user.favourites_count, tweet.user.followers_count, tweet.user.friends_count, 
                                      tweet.user.profile_image_url])

                        tweet_mention_results.append([s, tweet.id,tweet.user.id, 'No Mentions'])
                
                
                else:
                    if tweet.entities['urls'] == []:
                        tweet_results.append([s, tweet.id, tweet.full_text, tweet.created_at, tweet.place, tweet.retweet_count, tweet.favorite_count, "NO URLS", query])
                
                        twitter_user_results.append([tweet.user.id, tweet.user.screen_name, tweet.user.name, tweet.user.description, 
                                      tweet.user.location, tweet.user.created_at, tweet.user.favourites_count, tweet.user.followers_count, tweet.user.friends_count, 
                                      tweet.user.profile_image_url])
                
                        tweet_mention_results.append([s, tweet.id,tweet.user.id, tweet.entities['user_mentions'][0]['screen_name']])
                
                    else:
                        tweet_results.append([s, tweet.id, tweet.full_text, tweet.created_at, tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.entities['urls'][0]['url'], query])
                
                        twitter_user_results.append([tweet.user.id, tweet.user.screen_name, tweet.user.name, tweet.user.description, 
                                      tweet.user.location, tweet.user.created_at, tweet.user.favourites_count, tweet.user.followers_count, tweet.user.friends_count, 
                                      tweet.user.profile_image_url])
                
                        tweet_mention_results.append([s, tweet.id, tweet.user.id, tweet.entities['user_mentions'][0]['screen_name']])
                
    return(tweet_results,twitter_user_results,tweet_mention_results)


In [4]:
## Function to search all users and pull their tweets over the last 24 hours. Returns same info as tweet table in other func

def user_timeline(user_names):
    user_tweets = []
    
    print(user_names)
    
    now = datetime.datetime.today()
    date_since = datetime.datetime.strptime(time.strftime("%Y-%m-%d"), "%Y-%m-%d")
    day_ago_pre = now - datetime.timedelta(hours=24)
    day_ago = day_ago_pre.replace(tzinfo=pytz.utc)
    
    for name in user_names:
        count = 0
        for tweet in tweepy.Cursor(api.user_timeline, screen_name = name,
                                  tweet_mode = 'extended').items():
            
            if tweet.full_text.startswith('RT @'):
                pass
            
            elif (tweet.created_at < day_ago):
                break
            
            else:
                count = count + 1
                if tweet.entities['urls'] == []:
                    user_tweets.append([count, tweet.id, tweet.full_text, tweet.created_at, tweet.place, tweet.retweet_count, tweet.favorite_count, 'NO URLS', tweet.user.id])
                else:
                    user_tweets.append([count, tweet.id, tweet.full_text, tweet.created_at, tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.entities['urls'][0]['url'], tweet.user.id])
        
    return(user_tweets)

# Once Functions are completed, Data is Collected as seen below:

In [5]:
## Works 100% with few keywords, Rate Limit Not accounted for Yet
keywords_list = ["following roles sports", "SoccerJobs", "FootballJobs", "BasketballJobs", "BaseballJobs", "Job Opportunity Sports","Job Alert MLS", "Job Alert NFL", "Job Alert NBA", "Job Alert MLB", "Job Alert Premier League"]
tweet_search_rs,tweet_user_rs, tweet_mention_rs = search_tweets(keywords_list)

# Other keywords to try if time is not sensitive: Add spaces for Sport Jobs and "Jobs in Sports"

In [6]:
Tweets_df = pd.DataFrame(tweet_search_rs, columns = ['index', 'tweet_ID', 'tweet_contents', 'tweet_date_time', 'tweet_location','tweet_rt_count','tweet_fav_count', 'tweet_urls', 'keywords'])

Tweet_User_df = pd.DataFrame(tweet_user_rs,columns = ['user_ID', 'user_handle','user_name','user_bio','user_location','user_join_date','user_fav_count','user_follower_count','user_following_count','twitter_profile_img_url'])

Tweet_Mentions_df = pd.DataFrame(tweet_mention_rs, columns = ['mention_row_ID','tweet_ID', 'source_user_ID', 'mentioned_user'])

Tweets_df['tweet_user_ID'] = (Tweet_User_df['user_ID'])

In [7]:
user_tweets_rs = user_timeline(Tweet_User_df.user_handle.unique())


['apache14' 'MuskegonChron' 'Kzoo_Gazette' 'MLive' 'flintjournal'
 'JacksonCitPat' 'GRPressSports' 'annarbornews' 'GRPress'
 'WolverinesMLive' 'BayCityTimes' 'SpartansMLive' 'saginawnews'
 'MLiveSports' 'GiveNgosport' 'extrasoccernet' 'TeamWorkOnline'
 'JCPBedsAndHerts' 'Britcoachesnet' 'analyisport' 'Jobs4football'
 'PodyumRecruit' 'JCPinKent' 'JCPinSussex' 'SportingYF' 'minervapunjabfc'
 'SHPine1' 'KrisGoodman_' 'Justinsua' 'sandcvacancies' 'drivelinebases'
 'DrivelineBB' 'RaysRadio' 'jnorris427' 'BamaParrott' 'tmj_IDA_HRTA'
 'BWW_Careers' 'tmj_HOU_adm' 'JobsInSports' 'kimmy_sokol' 'admin_job'
 'rao_satyendra' 'ByaruhangaNass5' 'MikeZeigle' 'tmj_IDS_HRTA'
 'theaspire_group' 'ReboundJobs' 'tmj_IDN_HRTA' 'comminternships'
 'KMSanfeliz' '420JobsCareers' 'NBACram' 'warriorsviews' 'CelticsViews'
 'KingsharkSports' 'Sportekjobs']


In [8]:
User_Tweets_df = pd.DataFrame(user_tweets_rs, columns = ['index', 'tweet_ID', 'tweet_contents', 'tweet_date_time', 'tweet_location','tweet_rt_count','tweet_fav_count', 'tweet_urls', 'tweet_user_ID'])
User_Tweets_df['keywords'] = np.nan 

In [ ]:
Tweet_Mentions_df

In [9]:
User_Tweets_df = User_Tweets_df.reindex(columns = ['tweet_ID', 'tweet_user_ID', 'tweet_contents', 'tweet_date_time', 'tweet_location','tweet_rt_count','tweet_fav_count', 'tweet_urls', 'keywords'])
Tweets_df = Tweets_df.reindex(columns = ['tweet_ID', 'tweet_user_ID', 'tweet_contents', 'tweet_date_time', 'tweet_location','tweet_rt_count','tweet_fav_count', 'tweet_urls', 'keywords'])

In [10]:
Tweets_Final_df = pd.concat([Tweets_df,User_Tweets_df])

In [11]:
Tweets_Final_df['source_identifier'] = 1
Tweets_Final_df = Tweets_Final_df[(Tweets_Final_df.duplicated(['tweet_ID'])) == False]
Tweet_Mentions_df = Tweet_Mentions_df[(Tweet_Mentions_df.duplicated(['tweet_ID'])) == False]
Tweet_User_df = Tweet_User_df[(Tweet_User_df.duplicated(['user_ID'])) == False]

In [12]:
Sources = pd.DataFrame({'source_ID': [1], 'source_name': ['Twitter']})

# With the Data Collected, the database connection is established and all data is imported as seen below:

In [13]:
## Initialize connection to MYSQL
database = MySQLdb.connect(host="localhost" , user="root" , passwd="Pps11844")
cursor = database.cursor()

In [14]:
def execute_query(query_statement):
    try:
        cursor.execute(query_statement);
        database.commit();
        print("Data is Succefully Inserted")
    
    except Exception as e :
        database.rollback();
        print("The  Exception Occured : ", e)

In [15]:
execute_query(query_statement = ("CREATE Database IF NOT EXISTS JobsinSports"))

Data is Succefully Inserted


In [16]:
execute_query("USE JobsinSports")

Data is Succefully Inserted


In [ ]:
## Only used to delete and edit schema during Debugging phase 
#execute_query("DROP DATABASE JobsinSports")

In [17]:
execute_query("CREATE TABLE Tweets(tweet_ID BIGINT PRIMARY KEY NOT NULL UNIQUE, tweet_user_ID BIGINT NOT NULL, tweet_contents TEXT, tweet_date_time DATETIME, tweet_location TEXT, tweet_rt_count INT, tweet_fav_count INT, tweet_urls VARCHAR(255), keywords VARCHAR(100) DEFAULT NULL, source_identifier INT DEFAULT 0);")

Data is Succefully Inserted


In [18]:
execute_query("CREATE TABLE Sources(source_ID BIGINT PRIMARY KEY NOT NULL UNIQUE, source_name VARCHAR(55));")

Data is Succefully Inserted


In [19]:
execute_query("CREATE TABLE Tweet_User(user_ID BIGINT PRIMARY KEY NOT NULL UNIQUE, user_handle VARCHAR(25) NOT NULL UNIQUE, user_name VARCHAR(100), user_bio VARCHAR(255), user_location TEXT, user_join_date DATETIME, user_fav_count INT, user_follower_count INT, user_following_count INT, twitter_profile_img_url VARCHAR(255));")

Data is Succefully Inserted


In [20]:
execute_query("CREATE TABLE Tweet_Mentions(mention_row_ID INT PRIMARY KEY NOT NULL UNIQUE, source_tweet_ID BIGINT, source_user_ID BIGINT, mentioned_user VARCHAR(100));")

Data is Succefully Inserted


In [21]:
for i,j in Sources.iterrows():
    execute_query('INSERT INTO Sources (source_ID, source_name) VALUES (%d, "%s")' % (j['source_ID'],j['source_name']))


Data is Succefully Inserted


In [22]:
for i,j in Tweet_User_df.iterrows():
    print(i)
    execute_query('INSERT INTO Tweet_User (user_ID, user_handle, user_name, user_bio, user_location, user_join_date, user_fav_count, user_follower_count, user_following_count, twitter_profile_img_url) VALUES (%d, "%s", "%s", "%s", "%s", "%s", %d, %d, %d, "%s")' % (j['user_ID'], j['user_handle'], j['user_name'], j['user_bio'], j['user_location'], j['user_join_date'], j['user_fav_count'], j['user_follower_count'], j['user_following_count'], j['twitter_profile_img_url']))


0
Data is Succefully Inserted
1
Data is Succefully Inserted
2
Data is Succefully Inserted
3
Data is Succefully Inserted
4
Data is Succefully Inserted
5
Data is Succefully Inserted
6
Data is Succefully Inserted
7
Data is Succefully Inserted
8
Data is Succefully Inserted
9
Data is Succefully Inserted
10
Data is Succefully Inserted
11
Data is Succefully Inserted
12
Data is Succefully Inserted
13
Data is Succefully Inserted
14
Data is Succefully Inserted
15
Data is Succefully Inserted
60
Data is Succefully Inserted
62
Data is Succefully Inserted
64
Data is Succefully Inserted
65
Data is Succefully Inserted
68
Data is Succefully Inserted
83
Data is Succefully Inserted
87
Data is Succefully Inserted
88
Data is Succefully Inserted
113
Data is Succefully Inserted
135
Data is Succefully Inserted
169
Data is Succefully Inserted
197
Data is Succefully Inserted
198
Data is Succefully Inserted
200
Data is Succefully Inserted
208
Data is Succefully Inserted
212
Data is Succefully Inserted
213
Data i

In [23]:
# Tweets_df.shape

for i,j in Tweets_Final_df.iterrows():
    execute_query('INSERT INTO Tweets (tweet_ID, tweet_user_ID, tweet_contents, tweet_date_time, tweet_location, tweet_rt_count, tweet_fav_count, tweet_urls, keywords, source_identifier) VALUES (%d, %d, "%s", "%s", "%s", %d, %d, "%s", "%s", %d)' % (j['tweet_ID'], j['tweet_user_ID'], j['tweet_contents'], j['tweet_date_time'], j['tweet_location'],j['tweet_rt_count'],j['tweet_fav_count'], j['tweet_urls'], j['keywords'],j['source_identifier']))

Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully I

Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
The  Exception Occured :  (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'just a few generations, our family has gone from a lynching to law school, from ' at line 1")
Data is Succefully Inserted
Data is S

Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
The  Exception Occured :  (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'just a few generations, our family has gone from a lynching to law school, from ' at line 1")
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
The  Exception Occured :  (1064, 'You have an error in your SQL syntax; check the manual that

Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully I

Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully I

Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully I

Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully I

The  Exception Occured :  (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'I think it's just all part of natural evolution of a yo… #Rumors #GoldenState ' at line 1")
Data is Succefully Inserted
Data is Succefully Inserted
The  Exception Occured :  (1064, 'You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near \'He Could Take Over The Draymond Role..." #StrengthInNumbers #DubNation #GoldenSt\' at line 1')
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserte

In [24]:
for i,j in Tweet_Mentions_df.iterrows():
    execute_query('INSERT INTO Tweet_Mentions (mention_row_ID, source_tweet_ID, source_user_ID, mentioned_user) VALUES (%d, %d, %d,"%s")' % (j['mention_row_ID'],j['tweet_ID'], j['source_user_ID'], j['mentioned_user']))


Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully Inserted
Data is Succefully I

In [25]:
database.close()